<a href="https://colab.research.google.com/github/lrzcalmon/Repo_Git_Api/blob/main/Tech_Challenge_03_Llama_3_1_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine-tuning do Llama 3.1-8B em dados de produtos da Amazon

# Este notebook demonstra o processo de fine-tuning do modelo de linguagem Llama 3.1-8B usando a biblioteca Unsloth em dados de produtos da Amazon. O objetivo é adaptar o modelo para responder a perguntas com base nos títulos e conteúdos dos produtos.

## Estrutura do Notebook

# 1.  **Configuração Inicial:** Montagem do Google Drive e importação de bibliotecas essenciais.
# 2.  **Preparação do Dataset:** Carregamento, limpeza, normalização e formatação dos dados para o treinamento.
# 3.  **Instalação e Configuração do Modelo:** Instalação do Unsloth e configuração do modelo Llama 3.1-8B para fine-tuning com LoRA.
# 4.  **Treinamento do Modelo:** Definição do prompt, mapeamento do dataset e execução do treinamento.
# 5.  **Exportação do Modelo:** Salvamento do modelo treinado para uso futuro.
# 6.  **Inferência:** Carregamento do modelo fine-tuned e teste com exemplos de perguntas.


### Montagem do Google Drive

Montar o Google Drive com permissão de leitura e escrita no ambiente do Google Colab para permitir acessar e salvar arquivos diretamente do seu Drive, evitando a necessidade de carregar e baixar arquivos manualmente.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importação das bibliotecas necessárias para o processamento de dados.

In [2]:
import pandas as pd
import unicodedata
import json
import re

import os

# Desabilitar para não solicitar APIKey
os.environ["WANDB_DISABLED"] = "true"


### Preparação do Dataset

Esta seção descreve o processo de preparação dos dados brutos (em formato JSON line) para o formato adequado para o treinamento do modelo.

A função `prepara_dataset` realiza as seguintes etapas:

1.  **Carregamento:** Lê o arquivo JSON line e carrega os dados.
2.  **Seleção de Colunas:** Mantém apenas as colunas 'title' e 'content'.
3.  **Normalização:** Remove caracteres de controle e excesso de espaços.
4.  **Filtragem:** Remove registros com títulos ou conteúdos vazios após a normalização.
5.  **Remoção de Duplicados:** Remove registros duplicados com base em 'title' e 'content'.
6.  **Reset de Índice:** Reseta o índice do DataFrame após as operações de remoção.
7.  **Formatação para Treinamento:** Cria as colunas 'instruction', 'input' e 'output' no formato esperado pelo modelo (`instruction` para a tarefa, `input` para o título/pergunta, e `output` para o conteúdo/resposta).
8.  **Amostragem (Opcional):** Se especificado `tam_regs`, seleciona uma amostra aleatória do dataset.
9.  **Salvamento:** Salva o dataset formatado em um novo arquivo JSON.


In [4]:
# Efetua as diversas limpezas e estabilizações do dataset
def prepara_dataset(tam_regs=None, inpfile, outfile):
# Primeiro carrega o JSON como uma tupla no vetor data
    data = []
    with open(inpfile, 'r', encoding='utf-8') as file:
        for line in file: # para cada linha
            try:
                item = json.loads(line)
                if 'title' in item and 'content' in item:
                    title = item['title']
                    content = item['content']
                    if title or content is not None:
                        data.append({'title': title, 'content': content})  # só nos interessa titulo e conteudo
            except json.JSONDecodeError:
                pass # bypass de leituras erradas

    df = pd.DataFrame(data) # Usa o pandas para converter uma lista em dataFrame

    # Função para remover carac de controle e excesso de espaço para ganhar tokens
    def normalizar(text):
        text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != 'C')
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    # Aplica a função de normalização nas colunas 'title' e 'content'
    df['title'] = df['title'].apply(normalizar)
    df['content'] = df['content'].apply(normalizar)

    df = df[(df['title'].str.strip() != '') & (df['content'].str.strip() != '')] # remove brancos e vazios
    df.drop_duplicates(subset=['title', 'content'], inplace=True) # remove duplicados

    df.reset_index(drop=True, inplace=True) # como mexemos, precisa resetar o índice

    # Cria as colunas 'instruction', 'input' e 'output' para o formato necessário
    df['instruction'] = "Responda à pergunta com base no título do produto."
    df['input'] = df['title'].apply(lambda x: f"O que é '{x}'?")
    df['output'] = df['content']

    # Se tam_regs definir valor de corte
    if tam_regs is not None:
        df = df.sample(n=tam_regs, random_state=42).reset_index(drop=True)

    formatted_data = df[['instruction', 'input', 'output']].to_dict(orient='records') # Formata para dicionários

    with open(outfile, 'w', encoding='utf-8') as output_file:
        json.dump(formatted_data, output_file, ensure_ascii=False, indent=4) # Salva o Json para uso

    print(f"Total de registros processados: {len(df)}")

In [5]:
# Definindo os caminhos dos arquivos de entrada e saída
dataset_treino = "/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trn.json"
dataset_treino_output = "/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/formatted_train_dataset.json"

dataset_teste = "/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/tst.json"
dataset_teste_output = "/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/formatted_test_dataset.json"

Processando os datasets de treinamento e teste com o tamanho de registros especificado.




In [6]:
process_and_format_dataset(tam_regs=250000, dataset_treino, dataset_treino_output)  # 250.000 registros para treinamento
process_and_format_dataset(tam_regs=2500, teste_dataset, dataset_teste_output)     # 2.500 registros para teste

Dataset processado e salvo em '/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/formatted_train_dataset.json'. Total de registros: 250000
Dataset processado e salvo em '/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/formatted_test_dataset.json'. Total de registros: 2500


###Instalação do Unsloth e bibliotecas
Instala as bibliotecas necessárias para realizar o fine-tuning de forma eficiente em GPUs, como a Tesla T4, utilizando a biblioteca Unsloth. O Unsloth otimiza o processo de treinamento, tornando-o mais rápido e com menor consumo de memória.


In [7]:
!pip install "unsloth @ git+https://github.com/unslothai/unsloth.git" unsloth-zoo
!pip install peft accelerate bitsandbytes triton --no-deps xformers "trl<0.9.0" dataset

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-3i2ooyrd/unsloth_3d485ff4e58f4a5e9e59efba17613445
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-3i2ooyrd/unsloth_3d485ff4e58f4a5e9e59efba17613445
  Resolved https://github.com/unslothai/unsloth.git to commit 6e5534dc860005be7fd7790b3aa73c2a3996aa5e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


###Configuração do modelo


Vamos utilizar o modelo Llama 3.1, em GPUs com menos VRAM ou gratuitas como a **Tesla T4**.

In [9]:
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",           # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",         # We also uploaded 4bit for 405b!
]                                                   # More models at https://huggingface.co/unsloth

Carrega o modelo pré-treinado Llama 3.1-8B e o tokenizador correspondente. O modelo é carregado em 4 bits (quantização) para reduzir o consumo de memória VRAM, o que é crucial para GPUs com recursos limitados.


In [10]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch

max_seq_length = 128                        # Tamanho max da entrada.
model_name = "unsloth/Meta-Llama-3.1-8B"    # Modelo escolhido para executar o fine-tuning

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.6.0+cu124)
    Python  3.11.12 (you have 3.11.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.5: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### Configuração do PEFT e LoRA

Configura o modelo para treinamento eficiente utilizando a técnica LoRA (Low-Rank Adaptation of Lare Language Models). LoRA permite adaptar grandes modelos pré-treinados para tarefas específicas treinando apenas um pequeno número de parâmetros adicionais, o que reduz significativamente os requisitos computacionais  de memória.

Os parâmetros de configrção incluem:

*   `r`: Rank das matrizes LoRA. Um valor menor reduz o número de parâmetros treináveis e o custo cmputacional, mas pode limitar a capacidade de adaptação.
*   `target_modules`: Lista das camadas do modelo onde a adaptação LoRAserá aplicada.
*   `lora_alpha`: Escala das atulizações LoRA.
*   `lora_dropout`: Taxa de dropout aplicada às matrizes LoRA pararegularização.
*   `bias`: Configuração para o bias (geralmente 'none é otimizado).
*   `use_gradient_checkpointing`: Otimização de memória ativad pela Unsloth.
*   `random_state`: Semente para reprodutibilidade.


In [11]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,                                                     # Rank das matrizes LoRA podendo ser 8, 16, 32, 64, 128. Quanto menor o valor, menos custo computacional e menos poder de ajuste.

    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",   # Módulos do modelo que serão adaptados
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,                                            # Controla a escala de atualização das matrizes LoRA
    lora_dropout = 0,                                           # Ajuda a prevenir overfitting em tarefas específicas, onde 0 é otimizado e valores como 0.1 ou 0.2 podem ser úteis para baixo volume de dados.
    bias = "none",                                              # Nenhuma definição necessária para manter a otimização

    use_gradient_checkpointing = "unsloth",                     # Utilização de 30% menos de VRAM e 2x mais rápido para grandes contextos
    random_state = 3407,                                        # Número randômico

    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.5.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Formato do Prompt

Define o template do prompt que será utilizado para formatar as entradas e saídas do dataset para o treinamento. Este formato, baseado no template "Alpaca", ajuda o modelo a entender a estrutura da tarefa (instrução, entrada e resposta).



In [12]:
alpaca_prompt = """Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece contexto adicional. Escreva uma resposta que complete a solicitação adequadamente.

### instruction:
{}

### input:
{}

### output:
{}"""


### Aplicação do Formato do Prompt ao Dataset

Esta função aplica o template de prompt a cada exemplo no dataset. Para cada registro (que contém 'instruction', 'input' e 'output'), ela concatena esses campos no formato do `alpaca_prompt` e adiciona o `EOS_TOKEN` (End-of-Sequence token) ao final. O `EOS_TOKEN` indica ao modelo o fim de uma sequência.


In [13]:
EOS_TOKEN = tokenizer.eos_token            # Token que indica o fim de uma sequência
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Formata cada exemplo no template do prompt e adiciona o EOS_TOKEN.
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


### Inferência Antes do Fine-tuning

Demonstra como realizar a inferência com o modelo **antes** de aplicar o fine-tuning. Este teste mostra a capacidade do modelo base para responder a perguntas gerais. Observe que, como o modelo ainda não foi treinado nos dados específicos de produtos da Amazon, ele provavelmente não fornecerá respostas precisas ou relevantes para perguntas baseadas nesses dados.

Carrega o modelo e o tokenizador e os prepara para o modo de inferência. A função `FastLanguageModel.for_inference(model)` aplica otimizações específicas para geração de texto, tornando-a mais rápida.

Prepara o modelo para inferência com otimizações.


In [14]:
FastLanguageModel.for_inference(model) # Habilita inferência nativa que é 2x mais rápida


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

###Teste de prompt antes do fine-tuning com o dataset formatado


Prepara a entrada (prompt) para o modelo utilizando o tokenizador e especifica que o tensor retornado deve estar no dispositivo 'cuda' (GPU). Em seguida, gera texto com o modelo com base na entrada.


In [15]:
inputs = tokenizer(
[
    alpaca_prompt.format(
      "Responda à pergunta com base no título do produto.",  # instruction
      # "What is 'Girls Ballet Tutu Neon Pink'?",            # input
      # "Who was Ayrton Senna?",                             # input
      "Qual o último album do Pink FLoyd ?",                 # input
      "",                                                    # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda") # Retorna tensores PyTorch e move para a GPU.

outputs = model.generate(**inputs, max_new_tokens=200)  # Define o max de tokens gerados
# Decodifica a saída do modelo para texto legível, removendo tokens especiais.
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the product title.

### Input:
Last Pink FLoyd Album?

### Response:
Response to the request for a response to the request for a response to the request for a response to the request for a response to the request for a response to the request for a response to the request for a response to the request for a response to


### Carregamento do Dataset para Treinamento

Carrega o dataset de treinamento formatado (`formatted_train_dataset.json`) utilizando a biblioteca `datasets`. O dataset é carregado como um dataset do tipo "train". Em seguida, a função `formatting_prompts_func` é aplicada ao dataset para formatar cada exemplo de acordo com o template do prompt, criando a coluna 'text' que será usada para o treinamento.



Mapeamento do dataset formatado e gerado anteriormente pela preparação de dados.

In [16]:
from datasets import load_dataset
# Carrega o dataset formatado a partir do arquivo JSON.
dataset = load_dataset("json", data_files="/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/formatted_train_dataset.json", split="train")
# Aplica a função de formatação de prompts ao dataset.
dataset = dataset.map(formatting_prompts_func, batched = True,)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/250000 [00:00<?, ? examples/s]

### Treinamento do Modelo

Configura e executa o processo de fine-tuning do modelo utilizando o `SFTTrainer` da biblioteca `trl` (Transformers Reinforcement Learning). O `SFTTrainer` é otimizado para fine-tuning supervisionado em dados de texto formatados.

Os `TrainingArguments` definem os parâmetros do treinamento, incluindo:

*   `per_device_train_batch_size`: Tamanho do batch por GPU.
*   `gradient_accumulation_steps`: Número de passos antes de atualizar os gradientes.
*   `warmup_steps`: Passos iniciais para aquecimento da taxa de aprendizado.
*   `max_steps`: Número máximo de passos de treinamento.
*   `learning_rate`: Taxa de aprendizado.
*   `fp16` / `bf16`: Configurações de precisão.
*   `logging_steps`: Frequência de log.
*   `optim`: Otimizador (adamw_8bit é otimizado para memória).
*   `weight_decay`: Penalidade L2 para evitar overfitting.
*   `lr_scheduler_type`: Tipo de agendador de taxa de aprendizado.
*   `seed`: Semente para reprodutibilidade.
*   `output_dir`: Diretório para salvar resultados.
*   `report_to`: Configuração de relatório (desabilitado).
*   `save_strategy`: Estratégia de salvamento (desabilitado).

O `SFTTrainer` é então instanciado com o modelo, tokenizador, dataset de treinamento e os argumentos definidos. Finalmente, o método `trainer.train()` inicia o processo de fine-tuning.


In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Define o diretório onde os resultados do treinamento serão salvos.
output_dir = "/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M"

args = TrainingArguments(

    per_device_train_batch_size = 1,        # Tamanho do batch por GPU (reduzido para economizar memória).
    gradient_accumulation_steps = 2,        # Acumula gradientes por 2 passos antes de atualizar.
    warmup_steps = 5,                       # 5 passos de aquecimento da taxa de aprendizado.
    # num_train_epochs = 1,                 # Número de épocas (desabilitado, usando max_steps).
    max_steps = 60,                         # Treina por no máximo 60 passos.
    learning_rate = 2e-4,                   # Taxa de aprendizado inicial.
    fp16 = not is_bfloat16_supported(),     # Usa FP16 se bfloat16 não for suportado.
    bf16 = is_bfloat16_supported(),         # Usa BF16 se suportado (mais estável).
    logging_steps = 1,                      # Registra métricas a cada passo.
    optim = "adamw_8bit",                   # Otimizador AdamW otimizado para 8 bits.
    weight_decay = 0.01,                    # Penalidade L2.
    lr_scheduler_type = "linear",           # Agendador de taxa de aprendizado linear.
    seed = 3407,                            # Semente para reprodutibilidade.
    output_dir = output_dir,                # Diretório de saída.
    report_to = "none",                     # Não envia relatórios para plataformas externas.
    save_strategy="no",                     # Não salva checkpoints intermediários.
)

trainer = SFTTrainer(
    model = model,                          # Modelo a ser treinado.
    tokenizer = tokenizer,                  # Tokenizador do modelo.
    train_dataset = dataset,                # Dataset de treinamento formatado.
    dataset_text_field = "text",            # Campo no dataset que contém o texto formatado.
    max_seq_length = max_seq_length,        # Comprimento máximo da sequência.
    dataset_num_proc = 2,                   # Número de processos para carregar dados.
    packing = False,                        # Desabilita packing (pode acelerar para sequências curtas).
    args = args,                            # Argumentos de treinamento.
)

trainer.train()

Map (num_proc=2):   0%|          | 0/250000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 250,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,0.274300
2,0.377000
3,0.369800
4,0.487200
5,0.181600
6,0.130200
7,0.073100
8,0.095200
9,0.071600
10,0.053900


TrainOutput(global_step=60, training_loss=0.04051717385752151, metrics={'train_runtime': 98.2267, 'train_samples_per_second': 1.222, 'train_steps_per_second': 0.611, 'total_flos': 629363485974528.0, 'train_loss': 0.04051717385752151, 'epoch': 0.00048})

### Exportação do Modelo Treinado

Após o treinamento, é importante salvar o modelo fine-tuned e o tokenizador para que possam ser carregados e utilizados posteriormente sem a necessidade de refazer todo o processo de treinamento. Esta seção salva o modelo e o tokenizador no diretório especificado no Google Drive.


Salva os pesos do modelo fine-tuned e o tokenizador usado.

In [18]:
model.save_pretrained("/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trained_model")
tokenizer.save_pretrained("/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trained_model")

('/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trained_model/tokenizer_config.json',
 '/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trained_model/special_tokens_map.json',
 '/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trained_model/tokenizer.json')

### Utilização do Modelo com Fine-tuning (Inferência)

Esta seção demonstra como carregar o modelo que foi treinado com fine-tuning e utilizá-lo para realizar inferências (gerar respostas) para novas perguntas. O modelo carregado já incorpora o conhecimento adquirido durante o treinamento no dataset de produtos da Amazon.


Primeiro, instala a biblioteca Unsloth (caso esteja executando em uma nova sessão ou ambiente) e importa o `FastLanguageModel`.


In [19]:
!pip install unsloth
from unsloth import FastLanguageModel

# Em seguida, especifica o caminho para o modelo treinado salvo no Google Drive. Carrega o modelo e o tokenizador utilizando o `FastLanguageModel.from_pretrained()`, garantindo que sejam carregados com as mesmas configurações de quantização (4 bits) e comprimento máximo de sequência utilizadas no treinamento. Finalmente, prepara o modelo para o modo de inferência com as otimizações da Unsloth.
# Define o caminho para o modelo treinado salvo.
model_name = "/content/drive/MyDrive/TCC3-Fiap/LF-Amazon-1.3M/trained_model"

# Define as configurações para carregar o modelo (devem ser as mesmas do treinamento).
max_seq_length = 128                                  # Define o comprimento máximo das sequências de entrada.
dtype = None                                          # Tipo de dados (também pode ser torch.float16 ou torch.float32).
load_in_4bit = True                                   # Indica que o modelo será quantizado em 4 bits para economizar memória.

# Carrega o modelo e o tokenizador treinados.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    device_map="auto",
)

# Prepara o modelo para inferência (modo de avaliação e otimizações).
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.5.5: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

Prompt depois de realizer o fine-tuning do modelo que encontrará a resposta correta para a pergunta do usuário.

In [26]:
# Prepara a entrada para o modelo fine-tuned.
inputs = tokenizer(
[
    alpaca_prompt.format(
      "Responda à pergunta com base no conteudo do produto.",  # instruction
      #"What is 'Girls Ballet Tutu Neon Pink'?",           # input
      #"Who was Ayrton Senna?",                            # input
      "Último album do pink floyd ?",                           # input

      "",                                                 # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")                   # Retorna tensores PyTorch e move para a GPU.

outputs = model.generate(**inputs, max_new_tokens=500)   # Gera no máximo 500 novos tokens.
# Decodifica a saída do modelo para texto legível.
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the product content.

### Input:
Last Album?

### Response:
### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###